<a href="https://colab.research.google.com/github/LarissaCarrara/LarissaCarrara.github.io/blob/main/SINALIA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
%pip -q install google-genai

In [4]:
# Configura a API Key do Google Gemini

import os
from google.colab import userdata

os.environ["GOOGLE_API_KEY"] = userdata.get('GOOGLE_API_KEY')

In [5]:
# Instalar Framework de agentes do Google
!pip install -q google-adk

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 19.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.1/232.1 kB 16.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 217.1/217.1 kB 15.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 334.1/334.1 kB 19.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 125.1/125.1 kB 9.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.8/65.8 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.0/119.0 kB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.9/194.9 kB 13.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.3/103.3 kB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.

In [6]:
from google.adk.agents import Agent
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService
from google.adk.tools import google_search
from google.genai import types  # Para criar conteúdos (Content e Part)
from datetime import date
import textwrap # Para formatar melhor a saída de texto
from IPython.display import display, Markdown # Para exibir texto formatado no Colab
import requests # Para fazer requisições HTTP
import warnings

warnings.filterwarnings("ignore")

In [7]:
# Função auxiliar que envia uma mensagem para um agente via Runner e retorna a resposta final
def call_agent(agent: Agent, message_text: str) -> str:
    # Cria um serviço de sessão em memória
    session_service = InMemorySessionService()
    # Cria uma nova sessão (você pode personalizar os IDs conforme necessário)
    session = session_service.create_session(app_name=agent.name, user_id="user1", session_id="session1")
    # Cria um Runner para o agente
    runner = Runner(agent=agent, app_name=agent.name, session_service=session_service)
    # Cria o conteúdo da mensagem de entrada
    content = types.Content(role="user", parts=[types.Part(text=message_text)])

    final_response = ""
    # Itera assincronamente pelos eventos retornados durante a execução do agente
    for event in runner.run(user_id="user1", session_id="session1", new_message=content):
        if event.is_final_response():
          for part in event.content.parts:
            if part.text is not None:
              final_response += part.text
              final_response += "\n"
    return final_response

In [8]:
# Função auxiliar para exibir texto formatado em Markdown no Colab
def to_markdown(text):
  text = text.replace('•', '  *')
  return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

In [37]:
##########################################
# --- Agente: Busca a distancia entre os endereços --- #
##########################################
def agente_buscador(frase):

    buscador = Agent(
        name="agente_buscador",
        model="gemini-2.0-flash",
        instruction="""
       Você é um **Guia de Libras**. Sua tarefa é **traduzir frases do português para Libras**, **explicar como fazer os gestos**, e **fornecer um link** para ver o sinal em ação.

Use a ferramenta `Google Search` com esta estratégia:
1.  **Primeiro, tente buscar o sinal em Libras para a frase COMPLETA** que o usuário pedir. Priorize resultados que ofereçam vídeos ou imagens claras de como fazer o sinal da frase.
2.  **Se não encontrar o sinal para a frase completa**, então **quebre a frase em palavras-chave** e busque o sinal em Libras e um link para **CADA UMA DESSAS PALAVRAS**. Priorize vídeos ou imagens para cada palavra. Lembre-se que a estrutura da Libras pode ser diferente do português, então os sinais das palavras individuais são um ótimo ponto de partida.

Sua resposta DEVE ser muito útil e incluir:
* A **frase original** em português.
* **SE o sinal da frase completa for encontrado:**
    * Uma **descrição detalhada e clara de como fazer os gestos** do sinal da frase (movimentos, posição das mãos, expressão facial, etc.).
    * **O link** para o vídeo válido e disponível ou imagem que mostra o sinal da frase.
* **SE o sinal da frase completa NÃO for encontrado (e você buscar por palavras):**
    * Informe que, como a frase inteira não foi encontrada, você fornecerá os sinais das palavras-chave.
    * Para CADA palavra-chave encontrada, forneça:
        * A **palavra original**.
        * Uma **descrição clara de como fazer os gestos** do sinal daquela palavra.
        * **O link** para o vídeo ou imagem que mostra o sinal daquela palavra.
* **Se não encontrar nenhum link ou descrição clara (nem para a frase, nem para palavras) após todas as tentativas**, diga de forma educada que não foi possível encontrar um recurso visual específico e sugira buscar por um instrutor de Libras para um aprendizado mais aprofundado.
        """,
        description="A gente para analisar a distancia do trajeto",
        tools=[google_search]
    )

    entrada_do_agente_buscador = f"Frase para passar para libras: {frase}"

    distancias = call_agent(buscador, entrada_do_agente_buscador)
    return distancias

In [41]:
aq = input("Qual palavra/frase você gostaria de aprender em libras?: ")
result = agente_buscador(aq)
print("\n--- 📝 Resultado do Agente 1 (Buscador) ---\n")
display(to_markdown(result))

Qual palavra/frase você gostaria de aprender em libras?: oi, tudo bem?

--- 📝 Resultado do Agente 1 (Buscador) ---



> Olá! Para te ajudar a traduzir "Oi, tudo bem?" para Libras, faremos o seguinte:
> 
> 
> Como não encontrei um sinal único para toda a frase "Oi, tudo bem?", vou te mostrar como sinalizar cada parte:
> 
> **1. Oi:**
> 
> *   **Descrição:** Você pode usar o sinal da letra "O" e da letra "I" da Libras juntas em um movimento, ou simplesmente acenar com a mão, como um "oi" tradicional.
> *   **Vídeo:** Veja este vídeo para aprender o sinal de "Oi" em Libras: [https://www.youtube.com/watch?v=J5VnzXBxLIA](https://www.youtube.com/watch?v=J5VnzXBxLIA) ou [https://www.youtube.com/watch?v=4ZMLFWkwrf8](https://www.youtube.com/watch?v=4ZMLFWkwrf8)
> 
> **2. Tudo bem?:**
> 
> *   **Descrição:** Faça o sinal de "bom" (mão espalmada, com a palma para cima, e movimento para frente a partir do queixo), seguido do sinal de "joinha" (polegar para cima). A expressão facial deve acompanhar, demonstrando que você está perguntando se a pessoa está bem.
> *   **Vídeo:** Veja este vídeo para aprender o sinal de "Tudo bem?" em Libras: [https://www.youtube.com/watch?v=J5VnzXBxLIA](https://www.youtube.com/watch?v=J5VnzXBxLIA) ou [https://www.youtube.com/watch?v=EUyrmuJ1z0o](https://www.youtube.com/watch?v=EUyrmuJ1z0o)
> 
> Lembre-se que a Libras é uma língua viso-espacial. Além dos sinais com as mãos, a expressão facial e o movimento do corpo são muito importantes para transmitir a mensagem corretamente.
